In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
pd.set_option("display.max_rows", 4000)

In [3]:
url = 'https://townhall.com/election/{}/president/{}/county'
res = requests.get(url .format(2008, 'al'))
soup = BeautifulSoup(res.content, 'lxml')

In [4]:
rows = soup.find_all('tbody')[1]

In [5]:
results = pd.DataFrame()

In [6]:
# set list of states to interate over all counties in all states, missing alaska
states = ['al','az','ar','ca','co','ct','de','fl','ga','hi','id','il','in','ia','ks',
          'ky','la','me','md','ma','mi','mn','ms','mo','mt','ne','nv','nh','nj','nm','ny',
          'nc','nd','oh','ok','or','pa','ri','sc','sd','tn','tx','ut','vt','va','wa','wv',
          'wi','wy']

# set url of site to scrap from
url = 'https://townhall.com/election/{}/president/{}/county'

# lists of candidates for each year, to match election year entered into function
candidates_2004 = ['George Bush', 'John Kerry', 'Ralph Nader']
candidates_2008 = ['Barack Obama', 'John McCain']
candidates_2012 = ['Barack Obama', 'Mitt Romney', 'Gary Johnson', 'Other']
candidates_2016 = ['Hillary Clinton', 'Donald Trump', 'Gary Johnson', 'Jill Stein']


def data(year, candidates):
    '''Function that scrapes the site townhall.com for election data by county. Loops over 
    each state's page and outputs data for election year entered.'''
    
    # initialize empty lists
    rep_vote = []
    dem_vote = []
    counties = []
    
    # scrapes each state's page using list of states from above
    for state in states:
        res = requests.get(url .format(year, state))
        soup = BeautifulSoup(res.content, 'lxml')
        
        
        rows = soup.find_all('tbody')
        
        if len(rows) == 2:
            rows = soup.find_all('tbody')[1]
        else:
            rows = soup.find_all('tbody')

        # loops over rows to pull out county names
        for row in rows:
            count = []
            x = row.find('div')
            # filters returns from div returns three types of values (the county name, -1 and None) 
            # then appends to empty list to create list of lists, which will later be flattend
            if type(x) != int and x != None:
                county = x.text
                count.append(county)
                count = list(set(count))
                count.sort()
                count = [str(x + ', ' + state.upper()) for x in count]
                counties.append(count)

        
        # Republican Results - finds data from text with 'republican' class
        # finds class of REP and GOP, different for different years
        
        
        row = rows.find_all('td', {'class':  ['REP', 'GOP']})

        for x in row:
            if '.' not in x.text and x.text not in candidates:
                rep_vote.append(x.text.replace(',', ''))

       # Democratic Results - finds data from text with 'democratic' class

        row = rows.find_all('td', {'class': 'DEM'})
        

        for x in row:
            if '.' not in x.text and x.text not in candidates:
                dem_vote.append(x.text.replace(',', ''))
        
    return counties, dem_vote, rep_vote

## 2004 Election

In [7]:
count, dem_vote, rep_vote = data(2004, candidates_2004)

# check to make sure lists are the same length
print(len(count), len(dem_vote), len(rep_vote))

# loop over counties list to convert from list of lists or strings, to list of strings
counties = [item for sublist in count for item in sublist]

# empty DataFrame
results_2004 = pd.DataFrame()

# add lists to empty Data Frame
results_2004['counties'] = counties
results_2004['dem_vote'] = dem_vote
results_2004['rep_vote'] = rep_vote

results_2004

3110 3110 3110


counties dem_vote rep_vote
0                  Autauga, AL     4774    15212
1                  Baldwin, AL    15579    52910
2                  Barbour, AL     4826     5893
3                     Bibb, AL     2089     5471
4                   Blount, AL     3932    17364
5                  Bullock, AL     3210     1494
6                   Butler, AL     3409     4978
7                  Calhoun, AL    15076    29806
8                 Chambers, AL     5346     7618
9                 Cherokee, AL     3036     5919
10                 Chilton, AL     3773    12824
11                 Choctaw, AL     3289     3889
12                  Clarke, AL     4627     6729
13                    Clay, AL     1893     4622
14                Cleburne, AL     1387     4362
15                  Coffee, AL     4477    13010
16                 Colbert, AL    10586    13179
17                 Conecuh, AL     2715     3266
18                   Coosa, AL     2055     2902
19               Covington, AL     3422    11108
20                Crenshaw, AL     1698     3774
21                 Cullman, AL     8022    26783
22                    Dale, AL     4479    13614
23                  Dallas, AL    11145     7318
24                  DeKalb, AL     7092    16901
25                  Elmore, AL     6455    22024
26                Escambia, AL     3812     8501
27                  Etowah, AL    15311    26972
28                 Fayette, AL     2408     5532
29                Franklin, AL     4513     7688
30                  Geneva, AL     2105     8324
31                  Greene, AL     3763      958
32                    Hale, AL     4622     3278
33                   Henry, AL     2432     4859
34                 Houston, AL     9141    26868
35                 Jackson, AL     8618    11507
36               Jefferson, AL   132145   158504
37                   Lamar, AL     1956     4894
38              Lauderdale, AL    14615    22123
39                Lawrence, AL     6153     7728
40                     Lee, AL    16177    27911
41               Limestone, AL     8859    18994
42                 Lowndes, AL     4233     1786
43                   Macon, AL     7495     1540
44                 Madison, AL    52614    77150
45                 Marengo, AL     5026     5242
46                  Marion, AL     3806     8983
47                Marshall, AL     8441    22745
48                  Mobile, AL    63614    91877
49                  Monroe, AL     3666     5831
50              Montgomery, AL    45127    44074
51                  Morgan, AL    14113    32435
52                   Perry, AL     3754     1733
53                 Pickens, AL     3915     5170
54                    Pike, AL     4323     7473
55                Randolph, AL     2817     6127
56                 Russell, AL     8372     8332
57                  Shelby, AL    14826    63326
58               St. Clair, AL     5449    23468
59                  Sumter, AL     4527     1880
60               Talladega, AL    11373    18326
61              Tallapoosa, AL     5449    12378
62              Tuscaloosa, AL    26420    42827
63                  Walker, AL     9012    19162
64              Washington, AL     3127     5042
65                  Wilcox, AL     3238     1778
66                 Winston, AL     2234     8052
67                  Apache, AZ    15082     8068
68                 Cochise, AZ    16219    24828
69                Coconino, AZ    26513    20619
70                    Gila, AZ     7107    10494
71                  Graham, AZ     3141     7302
72                Greenlee, AZ     1146     1899
73                  La Paz, AZ     1849     3158
74                Maricopa, AZ   403882   539776
75                  Mohave, AZ    16267    29608
76                  Navajo, AZ    14224    16474
77                    Pima, AZ   154291   138431
78                   Pinal, AZ    25652    34813
79              Santa Cruz, AZ     6909     4668
80                 Yavapai, AZ    30377    49675
81  

## 2008 Election

In [8]:
count, dem_vote, rep_vote = data(2008, candidates_2008)

# check to make sure lists are the same length
print(len(count), len(dem_vote), len(rep_vote))

# loop over counties list to convert from list of lists or strings, to list of strings
counties = [item for sublist in count for item in sublist]

# empty DataFrame
results_2008 = pd.DataFrame()

# add lists to empty Data Frame
results_2008['counties'] = counties
results_2008['dem_vote'] = dem_vote
results_2008['rep_vote'] = rep_vote

results_2008

3112 3112 3112


counties dem_vote rep_vote
0                  Autauga, AL     6091    17398
1                  Baldwin, AL    19362    61192
2                  Barbour, AL     5685     5862
3                     Bibb, AL     2289     6247
4                   Blount, AL     3518    20362
5                  Bullock, AL     4001     1389
6                   Butler, AL     4174     5472
7                  Calhoun, AL    16325    32326
8                 Chambers, AL     6782     8060
9                 Cherokee, AL     2299     7285
10                 Chilton, AL     3666    13934
11                 Choctaw, AL     3633     4220
12                  Clarke, AL     5907     7455
13                    Clay, AL     1722     4946
14                Cleburne, AL     1166     5204
15                  Coffee, AL     5068    14909
16                 Colbert, AL     9698    14729
17                 Conecuh, AL     3411     3461
18                   Coosa, AL     2269     3245
19               Covington, AL     3238    12431
20                Crenshaw, AL     1938     4316
21                 Cullman, AL     5855    28837
22                    Dale, AL     5257    13873
23                  Dallas, AL    13958     6791
24                  DeKalb, AL     5654    17951
25                  Elmore, AL     8268    25695
26                Escambia, AL     5176     9365
27                  Etowah, AL    13480    30562
28                 Fayette, AL     1988     5875
29                Franklin, AL     3469     8048
30                  Geneva, AL     2106     9314
31                  Greene, AL     4402      876
32                    Hale, AL     4969     3196
33                   Henry, AL     2990     5558
34                 Houston, AL    12194    29205
35                 Jackson, AL     6367    14068
36               Jefferson, AL   166015   149843
37                   Lamar, AL     1614     5419
38              Lauderdale, AL    13318    24050
39                Lawrence, AL     5159     9269
40                     Lee, AL    21410    32344
41               Limestone, AL     9530    23588
42                 Lowndes, AL     5447     1807
43                   Macon, AL     9444     1396
44                 Madison, AL    64062    86910
45                 Marengo, AL     5925     5511
46                  Marion, AL     2597     9530
47                Marshall, AL     7021    25680
48                  Mobile, AL    81741    97670
49                  Monroe, AL     5023     6173
50              Montgomery, AL    61999    41972
51                  Morgan, AL    13882    35986
52                   Perry, AL     4423     1676
53                 Pickens, AL     4584     5426
54                    Pike, AL     5856     7981
55                Randolph, AL     3062     7169
56                 Russell, AL    10078     8700
57                  Shelby, AL    20575    68945
58               St. Clair, AL     6088    27630
59                  Sumter, AL     5218     1723
60               Talladega, AL    13769    20109
61              Tallapoosa, AL     6052    13105
62              Tuscaloosa, AL    32738    45351
63                  Walker, AL     7418    20719
64              Washington, AL     3032     5592
65                  Wilcox, AL     4553     1849
66                 Winston, AL     1756     8099
67                  Apache, AZ    15141     8381
68                 Cochise, AZ    17944    27590
69                Coconino, AZ    27064    19449
70                    Gila, AZ     7566    13425
71                  Graham, AZ     3487     8375
72                Greenlee, AZ     1165     1711
73                  La Paz, AZ     1794     3302
74                Maricopa, AZ   542206   675027
75                  Mohave, AZ    21286    42729
76                  Navajo, AZ    14953    19199
77                    Pima, AZ   191465   168670
78                   Pinal, AZ    42905    57714
79              Santa Cruz, AZ     8680     4517
80                 Yavapai, AZ    34731    58043
81  

## 2012 Election

In [9]:
count, dem_vote, rep_vote = data(2012, candidates_2012)

# check to make sure lists are the same length
print(len(count), len(dem_vote), len(rep_vote))

# loop over counties list to convert from list of lists or strings, to list of strings
counties = [item for sublist in count for item in sublist]

# due to faulty data entry in townhall.com Hawaii's county Kalawao needs to be removed
# as well as the entries corresponding to Kauai, entered as '-', which offset data
del counties[518] 
del rep_vote[518]
del dem_vote[518]

rep_vote.remove('-') 
dem_vote.remove('-')

# empty DataFrame
results_2012 = pd.DataFrame()

# add lists to empty Data Frame
results_2012['counties'] = counties
results_2012['dem_vote'] = dem_vote
results_2012['rep_vote'] = rep_vote

results_2012

3113 3114 3114


counties dem_vote rep_vote
0                  Autauga, AL     6354    17366
1                  Baldwin, AL    18329    65772
2                  Barbour, AL     5873     5539
3                     Bibb, AL     2200     6131
4                   Blount, AL     2961    20741
5                  Bullock, AL     4058     1250
6                   Butler, AL     4367     5081
7                  Calhoun, AL    15500    30272
8                 Chambers, AL     6853     7596
9                 Cherokee, AL     2126     7494
10                 Chilton, AL     3391    13910
11                 Choctaw, AL     3785     4150
12                  Clarke, AL     6317     7463
13                    Clay, AL     1770     4802
14                Cleburne, AL      971     5269
15                  Coffee, AL     4899    14638
16                 Colbert, AL     9160    13931
17                 Conecuh, AL     3551     3434
18                   Coosa, AL     2188     3028
19               Covington, AL     3155    12148
20                Crenshaw, AL     2048     4326
21                 Cullman, AL     4855    27930
22                    Dale, AL     5283    13105
23                  Dallas, AL    14599     6284
24                  DeKalb, AL     5235    18316
25                  Elmore, AL     8940    26223
26                Escambia, AL     5476     9263
27                  Etowah, AL    12792    29102
28                 Fayette, AL     1803     6034
29                Franklin, AL     3171     7567
30                  Geneva, AL     2039     9173
31                  Greene, AL     4514      799
32                    Hale, AL     5401     3205
33                   Henry, AL     3046     5605
34                 Houston, AL    12348    29214
35                 Jackson, AL     5817    14422
36               Jefferson, AL   159658   141413
37                   Lamar, AL     1643     5453
38              Lauderdale, AL    12504    23902
39                Lawrence, AL     5055     8855
40                     Lee, AL    21274    32062
41               Limestone, AL     9809    25263
42                 Lowndes, AL     5747     1754
43                   Macon, AL     9045     1322
44                 Madison, AL    61802    90679
45                 Marengo, AL     6154     5321
46                  Marion, AL     2249     9696
47                Marshall, AL     6271    25785
48                  Mobile, AL    78417    94566
49                  Monroe, AL     4897     5730
50              Montgomery, AL    62993    38283
51                  Morgan, AL    13401    35309
52                   Perry, AL     4536     1504
53                 Pickens, AL     4448     5119
54                    Pike, AL     6009     7939
55                Randolph, AL     3076     7219
56                 Russell, AL    10495     8276
57                  Shelby, AL    19944    71243
58               St. Clair, AL     5790    29001
59                  Sumter, AL     5420     1585
60               Talladega, AL    13878    19217
61              Tallapoosa, AL     6252    12303
62              Tuscaloosa, AL    32003    45703
63                  Walker, AL     6551    21633
64              Washington, AL     2971     5749
65                  Wilcox, AL     4867     1676
66                 Winston, AL     1286     8310
67                  Apache, AZ    17147     8250
68                 Cochise, AZ    17377    28197
69                Coconino, AZ    26772    19720
70                    Gila, AZ     7293    12846
71                  Graham, AZ     3609     8076
72                Greenlee, AZ     1310     1590
73                  La Paz, AZ     1619     3205
74                Maricopa, AZ   532284   680089
75                  Mohave, AZ    18991    47901
76                  Navajo, AZ    16945    19880
77                    Pima, AZ   186456   165237
78                   Pinal, AZ    41428    58947
79              Santa Cruz, AZ     9486     4235
80                 Yavapai, AZ    32624    62385
81  

## 2016 Election

In [10]:
count, dem_vote, rep_vote = data(2016, candidates_2016)

# check to make sure lists are the same length
print(len(count), len(dem_vote), len(rep_vote))

# loop over counties list to convert from list of lists or strings, to list of strings
counties = [item for sublist in count for item in sublist]

# empty DataFrame
results_2016 = pd.DataFrame()

# add lists to empty Data Frame
results_2016['counties'] = counties
results_2016['dem_vote'] = dem_vote
results_2016['rep_vote'] = rep_vote

results_2016

3111 3111 3111


counties dem_vote rep_vote
0                  Autauga, AL     5908    18110
1                  Baldwin, AL    18409    72780
2                  Barbour, AL     4848     5431
3                     Bibb, AL     1874     6733
4                   Blount, AL     2150    22808
5                  Bullock, AL     3530     1139
6                   Butler, AL     3716     4891
7                  Calhoun, AL    13197    32803
8                 Chambers, AL     5763     7803
9                 Cherokee, AL     1524     8809
10                 Chilton, AL     2909    15068
11                 Choctaw, AL     3109     4102
12                  Clarke, AL     5712     7109
13                    Clay, AL     1234     5230
14                Cleburne, AL      684     5738
15                  Coffee, AL     4194    15825
16                 Colbert, AL     7296    16718
17                 Conecuh, AL     3069     3413
18                   Coosa, AL     1780     3376
19               Covington, AL     2379    13222
20                Crenshaw, AL     1663     4511
21                 Cullman, AL     3730    32734
22                    Dale, AL     4408    13798
23                  Dallas, AL    12826     5784
24                  DeKalb, AL     3682    21779
25                  Elmore, AL     8436    27619
26                Escambia, AL     4698    10282
27                  Etowah, AL    10350    32132
28                 Fayette, AL     1358     6705
29                Franklin, AL     2197     9466
30                  Geneva, AL     1522     9967
31                  Greene, AL     4006      838
32                    Hale, AL     4772     3172
33                   Henry, AL     2286     5623
34                 Houston, AL    10547    30567
35                 Jackson, AL     3663    16643
36               Jefferson, AL   151581   130614
37                   Lamar, AL     1035     5819
38              Lauderdale, AL     9877    27735
39                Lawrence, AL     3611    10732
40                     Lee, AL    20987    34321
41               Limestone, AL     9340    28824
42                 Lowndes, AL     4882     1751
43                   Macon, AL     7237     1394
44                 Madison, AL    62435    89199
45                 Marengo, AL     5607     5224
46                  Marion, AL     1432    11273
47                Marshall, AL     4913    29217
48                  Mobile, AL    68429    91087
49                  Monroe, AL     4310     5771
50              Montgomery, AL    58669    33928
51                  Morgan, AL    11216    37392
52                   Perry, AL     3823     1403
53                 Pickens, AL     3972     5454
54                    Pike, AL     5039     7681
55                Randolph, AL     2290     7697
56                 Russell, AL     9577     9210
57                  Shelby, AL    22927    72846
58               St. Clair, AL     5550    31579
59                  Sumter, AL     4739     1581
60               Talladega, AL    12108    20596
61              Tallapoosa, AL     5271    12967
62              Tuscaloosa, AL    31746    47701
63                  Walker, AL     4486    24208
64              Washington, AL     2366     6031
65                  Wilcox, AL     4329     1737
66                 Winston, AL      871     9225
67                  Apache, AZ    12196     5315
68                 Cochise, AZ    15291    25036
69                Coconino, AZ    25308    16573
70                    Gila, AZ     6746    13672
71                  Graham, AZ     3301     8025
72                Greenlee, AZ     1092     1892
73                  La Paz, AZ     1318     3381
74                Maricopa, AZ   549040   590465
75                  Mohave, AZ    16485    54656
76                  Navajo, AZ    15362    18165
77                    Pima, AZ   195649   147019
78                   Pinal, AZ    35737    53857
79              Santa Cruz, AZ    10684     3650
80                 Yavapai, AZ    29705    58862
81  

In [15]:
results_2004.to_csv('./results_2004')

In [16]:
results_2016.to_csv('./results_2008')

In [17]:
results_2016.to_csv('./results_2012')

In [18]:
results_2016.to_csv('./results_2016')